<a href="https://colab.research.google.com/github/vkt1414/Cloud-Resources-Workflows/blob/main/Notebooks/Totalsegmentator/slicerMappingsTotalSegmentator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json
import yaml
import os
import sys
from pathlib import Path
import time
import ast

In [2]:
curr_dir   = Path().absolute()
os.environ['TZ'] = 'US/Eastern'
time.tzset()
current_time = time.strftime('%a %b %d %H:%M:%S %Y', time.localtime())
print(current_time)
print("\nCurrent directory :{}".format( curr_dir))
print("Python version    :", sys.version.split('\n')[0])

Wed Apr 05 20:34:52 2023

Current directory :/content
Python version    : 3.9.16 (main, Dec  7 2022, 01:11:51) 


In [3]:
try:
  os.remove(f'{curr_dir}/TotalSegmentator.py')
except OSError:
  pass
!wget https://raw.githubusercontent.com/lassoan/SlicerTotalSegmentator/main/TotalSegmentator/TotalSegmentator.py



--2023-04-05 20:34:55--  https://raw.githubusercontent.com/lassoan/SlicerTotalSegmentator/main/TotalSegmentator/TotalSegmentator.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66062 (65K) [text/plain]
Saving to: ‘TotalSegmentator.py’

TotalSegmentator.py 100%[===================>]  64.51K  --.-KB/s    in 0.01s   

2023-04-05 20:34:56 (5.47 MB/s) - ‘TotalSegmentator.py’ saved [66062/66062]



In [14]:
with open('TotalSegmentator.py', 'r') as f:
    content = f.read()
    tree = ast.parse(content)

class TotalSegmentatorLogicFinder(ast.NodeVisitor):
    def __init__(self):
        self.result = None

    def visit_ClassDef(self, node):
        if node.name == 'TotalSegmentatorLogic':
            for body_node in node.body:
                if isinstance(body_node, ast.FunctionDef) and body_node.name == '__init__':
                    for expr in body_node.body:
                        if isinstance(expr, ast.Assign):
                            target = expr.targets[0]
                            if isinstance(target, ast.Attribute) and target.attr == 'totalSegmentatorLabelTerminology':
                                self.result = ast.literal_eval(ast.unparse(expr.value))
        self.generic_visit(node)

finder = TotalSegmentatorLogicFinder()
finder.visit(tree)
totalSegmentatorLabelTerminology = finder.result
data = []
for key, value in totalSegmentatorLabelTerminology.items():
    fields = value.split('~')
    row = {'Structure': key}
    for i, field in enumerate(fields):
        subfields = field.split('^')
        if i == 0:
            if len(subfields) > 1:
                row['SegmentAlgorithmName'] = subfields[0]
                row['SegmentAlgorithmType'] = subfields[1]
        elif i == 1:
            if len(subfields) > 2:
                row['SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyCategoryCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyCategoryCodeSequence.CodeMeaning'] = subfields[2]
        elif i == 2:
            if len(subfields) > 2:
                row['SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyTypeCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyTypeCodeSequence.CodeMeaning'] = subfields[2]
        elif i == 3:
            if len(subfields) > 2:
                row['SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyTypeModifierCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyTypeModifierCodeSequence.CodeMeaning'] = subfields[2]
    data.append(row)

df = pd.DataFrame(data)
df

,Structure,SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator,SegmentedPropertyCategoryCodeSequence.CodeValue,SegmentedPropertyCategoryCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning
0,spleen,SCT,123037004,Anatomical Structure,SCT,78961009,Spleen,,,
1,kidney_right,SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,24028007,Right
2,kidney_left,SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,7771000,Left
3,gallbladder,SCT,123037004,Anatomical Structure,SCT,28231008,Gallbladder,,,
4,liver,SCT,123037004,Anatomical Structure,SCT,10200004,Liver,,,
...,...,...,...,...,...,...,...,...,...,...
99,autochthon_left,SCT,123037004,Anatomical Structure,SCT,44947003,Erector spinae muscle,SCT,7771000,Left
100,autochthon_right,SCT,123037004,Anatomical Structure,SCT,44947003,Erector spinae muscle,SCT,24028007,Right
101,iliopsoas_left,SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,7771000,Left
102,iliopsoas_right,SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,24028007,Right


In [26]:
import requests
import pandas as pd

base_url = 'https://browser.ihtsdotools.org/snowstorm/snomed-ct/MAIN'
headers = { 'User-Agent': 'Python'}

# Replace this with the actual language reference set id for your desired language
language_refset = '900000000000509007'

# Replace these with the actual column names from your dataframe
sct_codes = ['SegmentedPropertyCategoryCodeSequence.CodeValue', 'SegmentedPropertyTypeCodeSequence.CodeValue', 'SegmentedPropertyTypeModifierCodeSequence.CodeValue']

for sct_code_column in sct_codes:
    # Create new columns for preferred and acceptable terms
    preferred_terms_column = sct_code_column + '.PreferredTerms'
    acceptable_terms_column = sct_code_column + '.AcceptableTerms'
    df[preferred_terms_column] = None
    df[acceptable_terms_column] = None
    
    # Get unique SCT codes in column
    unique_sct_codes = df[sct_code_column].unique()
    
    # Get preferred and acceptable terms for each unique SCT code
    sct_code_terms = {}
    for sct_code in unique_sct_codes:
        if pd.isnull(sct_code):
            # If SCT code is missing, set preferred and acceptable terms to empty arrays
            sct_code_terms[sct_code] = ([], [])
        else:
            # Get all descriptions for the given SCT code
            response = requests.get(f'{base_url}/descriptions?conceptId={sct_code}&limit=50', headers=headers)
            descriptions = response.json()['items']
            
            # Filter descriptions by acceptability and activity
            preferred_terms = [desc['term'] for desc in descriptions if desc['active'] and desc['acceptabilityMap'].get(language_refset) == 'PREFERRED']
            acceptable_terms = [desc['term'] for desc in descriptions if desc['active'] and desc['acceptabilityMap'].get(language_refset) == 'ACCEPTABLE']
            
            # Store terms for SCT code
            sct_code_terms[sct_code] = (preferred_terms, acceptable_terms)
    
    # Add arrays to dataframe
    for index, row in df.iterrows():
        sct_code = row[sct_code_column]
        preferred_terms, acceptable_terms = sct_code_terms[sct_code]
        df.at[index, preferred_terms_column] = preferred_terms
        df.at[index, acceptable_terms_column] = acceptable_terms

sct_code_column = 'SegmentedPropertyTypeModifierCodeSequence.CodeValue'
preferred_terms_column = sct_code_column + '.PreferredTerms'
acceptable_terms_column = sct_code_column + '.AcceptableTerms'

# Remove or replace unexpected values in the preferred terms column
for index, row in df.iterrows():
    sct_code = row[sct_code_column]
    if sct_code == '':
        # If SCT code is an empty string, set preferred and acceptable terms to empty arrays
        df.at[index, preferred_terms_column] = []
        df.at[index, acceptable_terms_column] = []

column_order = ['Structure',
                'SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator',
                'SegmentedPropertyCategoryCodeSequence.CodeValue',
                'SegmentedPropertyCategoryCodeSequence.CodeMeaning',
                'SegmentedPropertyCategoryCodeSequence.CodeValue.PreferredTerms',
                'SegmentedPropertyCategoryCodeSequence.CodeValue.AcceptableTerms',
                'SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator',
                'SegmentedPropertyTypeCodeSequence.CodeValue',
                'SegmentedPropertyTypeCodeSequence.CodeMeaning',
                'SegmentedPropertyTypeCodeSequence.CodeValue.PreferredTerms',
                'SegmentedPropertyTypeCodeSequence.CodeValue.AcceptableTerms',
                'SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator',
                'SegmentedPropertyTypeModifierCodeSequence.CodeValue',
                'SegmentedPropertyTypeModifierCodeSequence.CodeMeaning',
                'SegmentedPropertyTypeModifierCodeSequence.CodeValue.PreferredTerms',
                'SegmentedPropertyTypeModifierCodeSequence.CodeValue.AcceptableTerms']

new_column_names = {'SegmentedPropertyCategoryCodeSequence.CodeValue.PreferredTerms': 'SegmentedPropertyCategoryCodeSequence.CodeValue.SNOMEDCTPreferredTerms',
                    'SegmentedPropertyCategoryCodeSequence.CodeValue.AcceptableTerms': 'SegmentedPropertyCategoryCodeSequence.CodeValue.SNOMEDCTAcceptableTerms',
                    'SegmentedPropertyTypeCodeSequence.CodeValue.PreferredTerms': 'SegmentedPropertyTypeCodeSequence.CodeValue.SNOMEDCTPreferredTerms',
                    'SegmentedPropertyTypeCodeSequence.CodeValue.AcceptableTerms': 'SegmentedPropertyTypeCodeSequence.CodeValue.SNOMEDCTAcceptableTerms',
                    'SegmentedPropertyTypeModifierCodeSequence.CodeValue.PreferredTerms': 'SegmentedPropertyTypeModifierCodeSequence.CodeValue.SNOMEDCTPreferredTerms',
                    'SegmentedPropertyTypeModifierCodeSequence.CodeValue.AcceptableTerms': 'SegmentedPropertyTypeModifierCodeSequence.CodeValue.SNOMEDCTAcceptableTerms'}

# Rename the columns of the DataFrame
df = df.rename(columns=new_column_names)
df

<ipython-input-26-8478e306cb9b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[preferred_terms_column] = None
<ipython-input-26-8478e306cb9b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[acceptable_terms_column] = None


,Structure,SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator,SegmentedPropertyCategoryCodeSequence.CodeValue,SegmentedPropertyCategoryCodeSequence.CodeMeaning,SegmentedPropertyCategoryCodeSequence.CodeValue.SNOMEDCTPreferredTerms,SegmentedPropertyCategoryCodeSequence.CodeValue.SNOMEDCTAcceptableTerms,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodeValue.SNOMEDCTPreferredTerms,SegmentedPropertyTypeCodeSequence.CodeValue.SNOMEDCTAcceptableTerms,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodeValue.SNOMEDCTPreferredTerms,SegmentedPropertyTypeModifierCodeSequence.CodeValue.SNOMEDCTAcceptableTerms
0,spleen,SCT,123037004,Anatomical Structure,"[Body structure (body structure), Body structure]",[Body structures],SCT,78961009,Spleen,"[Splenic structure, Splenic structure (body st...",[Spleen],,,,[],[]
1,kidney_right,SCT,123037004,Anatomical Structure,"[Body structure (body structure), Body structure]",[Body structures],SCT,64033007,Kidney,"[Kidney structure (body structure), Kidney str...","[Renal structure, Kidney]",SCT,24028007,Right,"[Right (qualifier value), Right]","[Dextro, Right lateral, Rt - Right, Right side]"
2,kidney_left,SCT,123037004,Anatomical Structure,"[Body structure (body structure), Body structure]",[Body structures],SCT,64033007,Kidney,"[Kidney structure (body structure), Kidney str...","[Renal structure, Kidney]",SCT,7771000,Left,"[Left (qualifier value), Left]","[Left lateral, Left side, Lt - Left, Levo-]"
3,gallbladder,SCT,123037004,Anatomical Structure,"[Body structure (body structure), Body structure]",[Body structures],SCT,28231008,Gallbladder,"[Gallbladder structure (body structure), Gallb...","[Gallbladder, Gall bladder]",,,,[],[]
4,liver,SCT,123037004,Anatomical Structure,"[Body structure (body structure), Body structure]",[Body structures],SCT,10200004,Liver,"[Liver structure (body structure), Liver struc...",[Liver],,,,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,autochthon_left,SCT,123037004,Anatomical Structure,"[Body structure (body structure), Body structure]",[Body structures],SCT,44947003,Erector spinae muscle,"[Structure of erector spinae muscle, Structure...",[Erector spinae muscle],SCT,7771000,Left,"[Left (qualifier value), Left]","[Left lateral, Left side, Lt - Left, Levo-]"
100,autochthon_right,SCT,123037004,Anatomical Structure,"[Body structure (body structure), Body structure]",[Body structures],SCT,44947003,Erector spinae muscle,"[Structure of erector spinae muscle, Structure...",[Erector spinae muscle],SCT,24028007,Right,"[Right (qualifier value), Right]","[Dextro, Right lateral, Rt - Right, Right side]"
101,iliopsoas_left,SCT,123037004,Anatomical Structure,"[Body structure (body structure), Body structure]",[Body structures],SCT,68455001,Iliopsoas muscle,[Structure of iliopsoas muscle (body structure...,[Iliopsoas muscle],SCT,7771000,Left,"[Left (qualifier value), Left]","[Left lateral, Left side, Lt - Left, Levo-]"
102,iliopsoas_right,SCT,123037004,Anatomical Structure,"[Body structure (body structure), Body structure]",[Body structures],SCT,68455001,Iliopsoas muscle,[Structure of iliopsoas muscle (body structure...,[Iliopsoas muscle],SCT,24028007,Right,"[Right (qualifier value), Right]","[Dextro, Right lateral, Rt - Right, Right side]"


In [27]:
df.to_csv('slicerMappingsToSnomedMappings.csv')